In [25]:
set(["input_ids", "attention_mask", "target_ids", "target_attention_mask" ])

{'attention_mask', 'input_ids', 'target_attention_mask', 'target_ids'}

In [6]:
from babl.model.T5.data import convert_to_features, build_dataset, T2TDataCollator

from functools import partial 
from dataclasses import dataclass
from pprint import pprint 


f = '/home/nameduser/Code/babl/inputs/10k.jsonl'
ds = build_dataset(f)


@dataclass
class Args:
    input_max_len : int = 512
    output_max_len : int = 32
    model_name_or_path: str = "t5-small"

args = Args()

txt2feats = partial(convert_to_features, args=args)
# map convert_to_features batch wise
ds = ds.map(txt2feats, batched=True)
print("train_dataset: ")
pprint(ds)
# valid_dataset = valid_dataset.map(add_eos_to_examples, load_from_cache_file=False)


# set the tensor type and the columns which the dataset should return
columns = ["input_ids", "target_ids", "attention_mask", "target_attention_mask"]
ds.set_format(type="torch", columns=columns)


num valid: 3194


/home/nameduser/.local/share/virtualenvs/babl-qoEDH0A2/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2681: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/home/nameduser/.local/share/virtualenvs/babl-qoEDH0A2/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
100%|██████████| 4/4 [00:07<00:00,  1.89s/it]

train_dataset: 
Dataset(features: {'input_text': Value(dtype='string', id=None), 'target_text': Value(dtype='string', id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'target_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'target_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 3194)


In [9]:
# dir(ds)
import torch 

vds  = torch.load('/home/nameduser/Code/babl/inputs/valid_data.pt')

tds  = torch.load('/home/nameduser/Code/babl/inputs/train_data.pt')

vds.data

/tmp/ipykernel_1565776/4138220314.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vds  = torch.load('/home/nameduser/Code/babl/inputs/valid_data.pt')
/tmp/ipykernel_1565

pyarrow.Table
input_text: string
target_text: string
input_ids: list<item: int64>
  child 0, item: int64
attention_mask: list<item: int64>
  child 0, item: int64
target_ids: list<item: int64>
  child 0, item: int64
target_attention_mask: list<item: int64>
  child 0, item: int64
----
input_text: [["question: who won icc award 2008 for twenty20 international performance of the year  context: <Ul> <Li> Cricketer of the Year : Shivnarine Chanderpaul ( West Indies ) </Li> <Li> Test Player of the Year : Dale Steyn ( South Africa ) </Li> <Li> ODI Player of the Year : MS Dhoni ( India ) </Li> <Li> Twenty20 International Performance of the Year : Yuvraj Singh ( India ) </Li> <Li> Emerging Player of the Year : Ajantha Mendis ( Sri Lanka ) </Li> <Li> Associate Player of the Year : Ryan ten Doeschate ( Netherlands ) </Li> <Li> Umpire of the Year : Simon Taufel ( Australia ) </Li> <Li> Women 's Cricketer of the Year : Charlotte Edwards ( England ) </Li> <Li> Spirit of Cricket : Sri Lanka </Li> <Li>

In [19]:
tdl = T2TDataCollator()(tds)# .info
vdl= T2TDataCollator()(vds)


Collator:
 dict_keys(['input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'])


Collator:
 dict_keys(['input_ids', 'attention_mask', 'target_ids', 'target_attention_mask'])



In [79]:
from argparse import ArgumentParser
from pathlib import Path 


parser = ArgumentParser()



parser.add_argument('--input-max-len', default=128)
parser.add_argument('--model-name-or-path', default='t5-small')
parser.add_argument('--output-max-len', default=32)


model_name = [a.default  for a in parser._actions if "model-name-or-path" in "".join(a.option_strings)][0]

print(model_name)
# parser._actions[0].option_strings
# root = Path(__file__).parent.parent
# output_dir = root / "outputs"
# input_dir = root / "inputs"
# parser.add_argument('--output-dir', default=output_dir)
# parser.add_argument('--input-dir', default=input_dir)
# parser.add_argument("--root-dir", default=root)

t5-small


## Loading BERT

In [1]:
from babl.model.bert import tokenizer, model
model

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense

ImportError: cannot import name 'tokenizer' from 'babl.model.bert' (unknown location)

In [52]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

import torch 
from babl.model.T5.eval import clean 
from pprint import pprint 
from pathlib import Path 



model_path= Path('/home/nameduser/Code/babl/outputs/checkpoint-1')
tok_path=  model_path.parent


tok = T5Tokenizer.from_pretrained(tok_path)
m = T5ForConditionalGeneration.from_pretrained(model_path)


Predictor(tok, m, max_len =128)("How many countries are there in the United Kingdom?")


# print(inference())



'one'

dance='hey'


In [18]:
next(iter(tdl))

'input_ids'

# Dataset and loader

In [11]:
from torch.utils.data import Dataset, dataloader


class T2tDataset(Dataset):

    def __init__(self, args):
